In [1]:

"""
这里主要参考了KAG的内容对RAG和KG的处理流程进行关键部分的解读来实现的。
https://github.com/OpenSPG/KAG/tree/master/kag/solver/prompt/default
https://github.com/OpenSPG/KAG/blob/master/kag/solver/prompt/default/logic_form_plan.py

实现的要点：
1、COT （思维链）-- 思考过程， 这里主要是根据已有的query和已经获取答案的sub query问答对，思考下一个生成的问题。
2、Planning (任务规划) -- 规划任务，这里是根据原始的query规划所有的步骤，一步一步地来实现下一个问题的生成，一步一步规划内容。
3、Resoning + Action(推理 + 动作)---推理答案 （下一个思考的问题） + continuing 或者 finished

4、Few shot example learning --> Proompt: 在Prompt中添加了响应格式的示例， 这里是少样本学习。
"""

"""

这里的思想（通过规划plan来实现下一步的思考问题--【通过当前原始问题、已经生成的子问题+答案对（或者query与上下文文档对）、推理思考过程】）
这里有点类似于：plan-guided reasoning rag.

同时逐步获取其中思路的方法有点类似于Auto-RAG的做法：
https://mp.weixin.qq.com/s/I_-NDGzd7d8l4zjRfCsvDQ。
https://mp.weixin.qq.com/s/5WvlT1kl0hPBKPxhXX_qKA

"""

import json

In [2]:

from openai import OpenAI


openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)


In [3]:



import torch

template_zh = (
        "你是一个智能助手，擅长通过复杂的、多跳的推理帮助用户在多文档中获取信息。请理解当前已知信息与目标问题之间的信息差。"
        "你的任务是直接生成一个用于下一步检索的思考问题。"
        "不要一次性生成所有思考过程！\n[已知信息]： $memory\n[目标问题]：$instruction\n[你的思考]："
    )




In [4]:


def query_decomposition_stepbystep(query, sloved):

    system_prompt =  """你是一个用户查询拆分的专家，擅长针对KBQA和搜索领域的查询进行分解,擅长通过复杂的多跳的推理帮助用户分解问题。
    你的任务是直接生成一个用于下一步检索的思考问题。
    思考问题不要借助外部知识推理，仅根据问题进行思考。
    生成的思考问题独立明确，可直接回答或检索。
    直接生成要思考的问题，不要其它问题的结尾词。
    不要一次性生成所有的思考过程。

    原始问题：
    {query}
    已经思考过的问题和答案：
    {sloved_query}
    下一步思考的问题：
    
    """

    messages=[
        {"role": "system", "content": "您是一个有益的人工智能助手。"},
        {"role": "user", "content":  system_prompt.format(**{"query": query, "sloved_query": sloved})},
    ]
    chat_response = client.chat.completions.create(model="Qwen2.5-7B-Instruct",messages=messages, 
                                                   temperature=0.001,
                                                   # top_p=0.8,
                                                   max_tokens=1024,
                                                   extra_body={
                                                               "repetition_penalty": 1.05,
                                                       # "guide_json": QueryDecomposition.model_json_schema(),
                                                       # "guided_decoding_backend": "outlines"
                                                   }
                                                  )

    
    
    return chat_response.choices[0].message.content



In [5]:

def judge_finish_answer(query, sloved_sub_query):

    prompt = """ 跟定一个原始的用户问题和若干个已经拆分后的子问题及答案，判断是否能够回答原始的用户问题。
    如果能回答原始问题，则输出finished和原始问题的答案answer。
    如果不能回答原始，则输出continuing, 原始问题的答案answer为空。
    不要输出额外的内容。
    
    输出结果为json，格式样例如下：
    {{"judge": "finished", "answer": "原始问题的答案"}}
    {{"judge": "continuing", "answer": ""}}
    
    原始问题：
    {query}

    已有的问题答案对：
    {sloved_sub_query}
    """
    messages=[
        {"role": "system", "content": "您是一个有益的人工智能助手。"},
        {"role": "user", "content":  prompt.format(**{"query": query, "sloved_sub_query": sloved_sub_query})},
    ]
    chat_response = client.chat.completions.create(model="Qwen2.5-7B-Instruct",messages=messages, 
                                                   temperature=0.001,
                                                   # top_p=0.8,
                                                   max_tokens=1024,
                                                   extra_body={
                                                               "repetition_penalty": 1.05,
                                                       # "guide_json": QueryDecomposition.model_json_schema(),
                                                       # "guided_decoding_backend": "outlines"
                                                   }
                                                  )
    response = chat_response.choices[0].message.content
    response = response.replace("json", "").replace("```", "")
    
    return response



    

In [6]:
query = "姚明和奥尼尔谁的身高高？"
sloved_sub_query = "问题：姚明的身高是多少？ 答案：2.42m"
response = judge_finish_answer(query, sloved_sub_query)
print(response)

{"judge": "continuing", "answer": ""}


In [7]:


query = "姚明和奥尼尔谁的身高高？"
sloved = ""
response = query_decomposition_stepbystep(query, sloved)
print(response)


姚明的身高是多少？


In [8]:
def query_step_plan(query):

    sloved_answer = ""
    continuing = True
    sub_query_index = 1
    
    while continuing:
        sub_query = query_decomposition_stepbystep(query, sloved_answer)
        response = input(f"{sub_query_index}. {sub_query}:")
        sloved_answer += f"思考问题：{sub_query}, 答案：{response}" + "\n"
        judge_response = judge_finish_answer(query, sloved_answer)
        judge = json.loads(judge_response)["judge"]
        if judge == "finished":
            sub_query_index += 1
            print(str(sub_query_index) + ".", "最终答案：", json.loads(judge_response)["answer"])
            continuing = False
        
        else:
            # print("子问题的解答过程：", sloved_answer)
            sub_query_index += 1
            
    



In [10]:
query = "姚明和奥尼尔谁的身高高？"
query = "张三的父亲的母亲多大了"
query = "耗电量最大的设备是什么时候购买的"
query = "最后两届男子板球世界杯决赛的举办地之间的距离是多少"
query = "最后两届男子板球世界杯决赛的举办地分别在哪里？"
query_step_plan(query)

1. 最后一届男子板球世界杯决赛的举办地在哪里？: 上海
2. 倒数第二屆男子板球世界杯决赛的举办地在哪里？: 北京


3. 最终答案： 最后两届男子板球世界杯决赛的举办地分别在北京和上海。
